
# Preparation



Installing and importing libraries

In [ ]:
# install correct version of tensorflow
!pip install tensorflow==2.1
# install libraries if needed
!pip install sentence_transformers

     |████████████████████████████████| 421.8 MB 20 kB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 448 kB 44.1 MB/s 
     |████████████████████████████████| 3.8 MB 61.3 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=bbd134dc307254ed0e1f13f8c05b10f9656998df6340906ab05e9e28bbf76c7f
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

     |████████████████████████████████| 85 kB 1.9 MB/s 
     |████████████████████████████████| 2.8 MB 19.3 MB/s 
     |████████████████████████████████| 1.2 MB 53.7 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 636 kB 54.8 MB/s 
     |████████████████████████████████| 3.3 MB 22.3 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=b963ee66cba4b0201bab8636bf0940b868fdd597376b7319e1d866d262ba67b7
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from sentence_transformers import SentenceTransformer

Mount to Google drive and set Dataset paths

In [ ]:
# mount to Google drive
from google.colab import drive
drive.mount('/content/drive')

# dataset paths
dataset_path = '/content/drive/My Drive/spring_2021/questionaire_project/google_collab/dataset/'
responses_file_name = 'Suhaib/BIG5/big5_responses.csv'
output_embeddings_file_name = 'Suhaib/BIG5/big5_questions_embeddings_SENTENCEBERT.csv'

Mounted at /content/drive


Loading item text (output format should be consistent for both Big5 dataset and IPIP dataset)

In [ ]:
#load item-response matrix
matrix = pd.read_csv(dataset_path + responses_file_name, header = 0)
print(matrix[-5:])

# preprocess data
# ATTENTION: check if item has "I" or not! If not, manually add "I"
matrix['item'] = [(item).lower() for item in matrix['item']]

# extract data
question_ids = matrix['question-id']
individual_ys = matrix.drop(['question-id'], axis=1).set_index(["item"]).T
items = matrix.iloc[:,1].tolist()
print(items)
print(len(items))

   question-id  ... 74f7bd13c33957ecd985dc5cd951dd14
95         q96  ...                                5
96         q97  ...                                3
97         q98  ...                                4
98         q99  ...                                5
99        q100  ...                                1

[5 rows x 2751 columns]
['i have a vivid imagination', 'i hold a grudge', 'i do not mind being the centre of attention', 'i do not like poetry', 'i complete tasks successfully', 'i believe that others have good intentions', 'i avoid philosophical discussions', 'i need a push to get started', 'i cut others to pieces', 'i make friends easily', 'i feel comfortable with myself', 'i often feel blue', 'i am easy to satisfy', 'i keep in the background', 'i am always prepared', 'i enjoy wild flights of fantasy', 'i get stressed out easily', 'i avoid contact with others', 'i am not easily bothered by things', 'i shirk my duties', 'i can say things beautifully', 'i suspect hidden mo

# Extract embeddings

Extracting sentence-BERT embeddings.

In [ ]:
## sentence-BERT
model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
sentence_embeddings = model.encode(items)
vec = sentence_embeddings
print(vec.shape)

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

(100, 1024)


# Save to files

In [ ]:
print(vec.shape)
embeddings_df = pd.DataFrame(vec)
embeddings_df.insert(0,'question-id', question_ids)
embeddings_df.columns = ['question-id'] + list(range(vec.shape[1]))
print("saving to file: " + dataset_path + output_embeddings_file_name)
print(embeddings_df.head())
embeddings_df.to_csv(dataset_path + output_embeddings_file_name, index = False, header = True)

(32, 500)
saving to file: /content/drive/My Drive/spring_2021/questionaire_project/google_collab/dataset/Suhaib/HSQ/hsq_questions_embeddings_WORD2VEC.csv
  question-id         0         1  ...       497       498       499
0          Q1 -0.010657 -0.034137  ... -0.035971  0.169668  0.069747
1          Q2  0.004098  0.022988  ...  0.055723  0.109839  0.062696
2          Q3 -0.036123 -0.050522  ...  0.037835  0.146944  0.059633
3          Q4 -0.004308  0.034791  ...  0.045398  0.136608  0.069128
4          Q5 -0.021181 -0.056163  ...  0.001984  0.250811  0.133104

[5 rows x 501 columns]
